# Tokenizer

## 概述

为了方便后续向量化表征试题，本模块提供题目文本的令牌化解析（Tokenization），即将题目转换成令牌序列。    

根据构成题目的元素类型，解析功能分为 **“文本解析”** 和 **“公式解析”** 两部分。

### 文本解析

根据题目文本切分粒度的大小，文本解析又分为 **“句解析”** 和 **“词解析”**。

（1） 句解析（sentence-tokenization）：将较长的文档切分成若干句子的过程称为“分句”。每个句子为一个“令牌”（token）。（待实现）    
  

（2） 词解析（text-tokenization）：一个句子（不含公式）是由若干“词”按顺序构成的，将一个句子切分为若干词的过程称为“词解析”。根据词的粒度大小，又可细分为“词组解析”和"单字解析"。
- 词组解析 (word-tokenization)：每一个词组为一个“令牌”（token）。
- 单字解析 (char-tokenization)：单个字符即为一个“令牌”（token）。

### 公式解析

公式解析（formula-tokenization）：理科类文本中常常含有公式。将一个符合 latex 语法的公式切分为标记字符列表的过程称为“公式解析”。每个标记字符为一个“令牌”（token）。  
  

## 文本解析

### 句解析

待实现...

### 词解析

词解析分为两个主要步骤：  

（1） 分词：  
- 词组解析：使用分词工具切分并提取题目文本中的词。  
    本项目目前支持的分词工具有：`jieba`   
- 单字解析：按字符划分。
    
      
（2） 筛选：过滤指定的停用词。   
- 本项目默认使用的停用词表：[stopwords](https://github.com/bigdata-ustc/EduNLP/blob/master/EduNLP/meta_data/sif_stopwords.txt)  
- 你也可以使用自己的停用词表，具体使用方法见下面的示例。


In [1]:
# 导入模块
from EduNLP.SIF.tokenization.text import tokenize 

In [2]:
# 输入
text = "三角函数是基本初等函数之一"

#### 词组解析

分词粒度参数选择 word： `granularity = "word"` 

In [3]:
# 输出：默认使用 EduNLP 项目提供的停用词表
tokenize(text, granularity="word")

['三角函数', '初等', '函数']

#### 单字解析

分词粒度参数选择 word： `granularity = "char"` 

In [4]:
# 输出：默认使用 EduNLP 项目提供的停用词表
tokenize(text, granularity="char")

['三', '角', '函', '数', '基', '初', '函', '数']

#### 停用词表

In [5]:
# 获取自己的停用词表
spath = "test_stopwords.txt"
from EduNLP.SIF.tokenization.text.stopwords import get_stopwords
stopwords = get_stopwords(spath)
stopwords

{'一旦', '一时', '一来', '一样', '一次', '一片', '一番', '一直', '一致'}

In [6]:
# 输出：传入停用词表（stopwords）
tokenize(text,granularity="word",stopwords=stopwords)

['三角函数', '是', '基本', '初等', '函数', '之一']

## 公式解析
切分出 latex 公式的每个标记符号。针对本模块更加详细的解释参见 [formula](../formula/formula.ipynb)

In [7]:
# 导入模块
from EduNLP.SIF.tokenization.formula import tokenize

- 输入

In [8]:
formula = "\\frac{\\pi}{x + y} + 1 = x"

- 输出

（1）如果您想按 latex 语法标记拆分公式的各个部分，并得到顺序序列结果，输出方法可以选择：`linear`

In [9]:
tokenize(formula, method="linear")

['\\frac', '{', '\\pi', '}', '{', 'x', '+', 'y', '}', '+', '1', '=', 'x']

（2） 如果您想得到公式解析出的语法分析树序列，输出方法可以选择：`ast`
    > 抽象语法分析树，简称语法树（Syntax tree），是源代码语法结构的一种抽象表示。它以树状的形式表现编程语言的语法结构，树上的每个节点都表示源代码中的一种结构。  

    > 因此，ast 可以看做是公式的语法结构表征。

In [10]:
tokenize(formula, method="ast", return_type="list", ord2token=False)

['\\pi', '{ }', 'x', '+', 'y', '{ }', '\\frac', '+', '1', '=', 'x']

（3）如果您只是关心公式的结构和类型，并不关心变量具体是什么，比如二元二次方程 `x^2 + y = 1` ，它从公式结构和类型上来说，和 `w^2 + z = 1` 没有区别。  
此时，您可以设置如下参数：`ord2token = True`，将公式变量名转换成 token

In [11]:
# 输出形式选择抽象语法分析树（ast）且将公式变量名转换成 token
tokenize(formula, method="ast", return_type="list", ord2token=True)

['mathord',
 '{ }',
 'mathord',
 '+',
 'mathord',
 '{ }',
 '\\frac',
 '+',
 'textord',
 '=',
 'mathord']

（4） 如果您除了 （3） 中提供的功能之外，还需要区分不同的变量。此时可以另外设置参数：`var_numbering=True`

In [12]:
# 输出形式选择抽象语法分析树（ast）且将公式变量名转换成带编号的 token
tokenize(formula, method="ast", return_type="list", ord2token=True, var_numbering=True)

['mathord_con',
 '{ }',
 'mathord_0',
 '+',
 'mathord_1',
 '{ }',
 '\\frac',
 '+',
 'textord',
 '=',
 'mathord_0']

## 综合解析

综合解析，即综合以上两种解析方式（标记解析 + 公式解析），提供对题目文本的全解析。另外，如遇到特殊符号将转换成常量，例如：
```python
FIGURE_SYMBOL = "[FIGURE]" # $\SIFChoice$
QUES_MARK_SYMBOL = "[MARK]" # $\FigureID{1}$
```


In [39]:
# 导入模块
from EduNLP.Tokenizer import get_tokenizer

# 输入
item = {
    "如图来自古希腊数学家希波克拉底所研究的几何图形．此图由三个半圆构成，三个半圆的直径分别为直角三角形$ABC$的斜边$BC$, 直角边$AB$, $AC$.$\bigtriangleup ABC$的三边所围成的区域记为$I$,黑色部分记为$II$, 其余部分记为$III$.在整个图形中随机取一点，此点取自$I,II,III$的概率分别记为$p_1,p_2,p_3$,则$\SIFChoice$$\FigureID{1}$"
}

# 输出
tokenizer = get_tokenizer("text")
tokens = tokenizer(item)
next(tokens) 

['如图',
 '古希腊',
 '数学家',
 '希波',
 '克拉底',
 '研究',
 '几何图形',
 '此图',
 '三个',
 '半圆',
 '三个',
 '半圆',
 '直径',
 '直角三角形',
 'ABC',
 '斜边',
 'BC',
 '直角',
 'AB',
 'AC',
 '\x08',
 'igtriangleupABC',
 '三边',
 '围成',
 '区域',
 '记',
 'I',
 '黑色',
 '记',
 'II',
 '其余部分',
 '记',
 'III',
 '图形',
 '中',
 '随机',
 '取',
 '一点',
 '此点',
 '取自',
 'I',
 ',',
 'II',
 ',',
 'III',
 '概率',
 '记',
 'p',
 '_',
 '1',
 ',',
 'p',
 '_',
 '2',
 ',',
 'p',
 '_',
 '3',
 '[MARK]',
 '[FIGURE]']